In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
books = pd.read_csv('./Bookg.csv',encoding='latin-1')
users = pd.read_csv('./Users.csv', encoding='latin-1')
ratings = pd.read_csv('./Ratings.csv', encoding='latin-1')

In [3]:
books

,isbn,title,author,year,publisher,image_url,genres
0,399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,"['Fiction', 'Historical Fiction', 'China', 'As..."
1,440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,"['Fiction', 'Thriller', 'Mystery', 'Legal Thri..."
2,609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,"['Humor', 'History', 'Comedy', 'Nonfiction', '..."
3,345402871,Airframe,Michael Crichton,1997,Ballantine Books,http://images.amazon.com/images/P/0345402871.0...,"['Fiction', 'Thriller', 'Mystery', 'Science Fi..."
4,345417623,Timeline,MICHAEL CRICHTON,2000,Ballantine Books,http://images.amazon.com/images/P/0345417623.0...,"['Science Fiction', 'Fiction', 'Thriller', 'Ti..."
...,...,...,...,...,...,...,...
26784,1561569119,Robinson Crusoe,Daniel Defoe,1998,Kidsbooks,http://images.amazon.com/images/P/1561569119.0...,"['Classics', 'Fiction', 'Adventure', 'Literatu..."
26785,451521390,Pathfinder,James Fenimore Cooper,1982,Signet Book,http://images.amazon.com/images/P/0451521390.0...,"['Fantasy', 'Science Fiction', 'Young Adult', ..."
26786,1902852036,All the Queen's Men,Nick Elwood,1999,Gay Men's Press,http://images.amazon.com/images/P/1902852036.0...,"['Romance', 'Romantic Suspense', 'Suspense', '..."
26787,441011799,Singularity Sky,Charles Stross,2004,Ace,http://images.amazon.com/images/P/0441011799.0...,"['Science Fiction', 'Fiction', 'Space Opera', ..."


In [5]:
books.dtypes

isbn         object
title        object
author       object
year          int64
publisher    object
image_url    object
genres       object
dtype: object

In [6]:
books.shape

(26789, 7)

In [7]:
users

,user_id,location,age
0,1,"nyc, new york, usa;NULL",NaN
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia;NULL",NaN
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom;NULL",NaN
...,...,...,...
278847,278854,"portland, oregon, usa;NULL",NaN
278848,278855,"tacoma, washington, united kingdom",50
278849,278856,"brampton, ontario, canada;NULL",NaN
278850,278857,"knoxville, tennessee, usa;NULL",NaN


In [8]:
ratings

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6
...,...,...,...
1048570,250764,451410777,0
1048571,250764,452264464,8
1048572,250764,048623715X,0
1048573,250764,486256588,0


In [9]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: count, Length: 95513, dtype: int64

In [10]:
x = ratings['user_id'].value_counts() > 50
x[x].shape

(3066,)

In [11]:
y= x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       205197, 239486, 220911, 185731, 213119,  76440, 167521,  79506, 276847,
        39705],
      dtype='int64', name='user_id', length=3066)

In [12]:
ratings = ratings[ratings['user_id'].isin(y)]

In [13]:
ratings

,user_id,isbn,rating
173,276847,446364193,0
174,276847,3257200552,5
175,276847,3379015180,0
176,276847,3404145909,8
177,276847,3404148576,8
...,...,...,...
1048570,250764,451410777,0
1048571,250764,452264464,8
1048572,250764,048623715X,0
1048573,250764,486256588,0


In [14]:
ratings_with_books = ratings.merge(books, on='isbn')

In [15]:
ratings_with_books

,user_id,isbn,rating,title,author,year,publisher,image_url,genres
0,276925,140154078,6,The Music of Chance,Paul Auster,1993,Penguin Books,http://images.amazon.com/images/P/0140154078.0...,"['Fiction', 'American', 'Literature', 'Novels'..."
1,276925,140327592,0,Matilda,Roald Dahl,1990,Viking Penguin Inc,http://images.amazon.com/images/P/0140327592.0...,"['Childrens', 'Fiction', 'Classics', 'Fantasy'..."
2,276925,151010633,0,Q,Luther Blissett,2004,Harcourt,http://images.amazon.com/images/P/0151010633.0...,"['Historical Fiction', 'Fiction', 'Historical'..."
3,276925,385504209,8,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,"['Fiction', 'Mystery', 'Thriller', 'Suspense',..."
4,276925,553275283,8,Love Story,Erich Segal,1988,Bantam Books,http://images.amazon.com/images/P/0553275283.0...,"['Romance', 'Fiction', 'Classics', 'Contempora..."
...,...,...,...,...,...,...,...,...,...
168971,250764,451191692,0,Remote Control,Stephen White,1998,Signet Book,http://images.amazon.com/images/P/0451191692.0...,"['Thriller', 'Fiction', 'Action', 'Military Fi..."
168972,250764,451192001,0,How Stella Got Her Groove Back,Terry McMillan,1997,Signet Book,http://images.amazon.com/images/P/0451192001.0...,"['Fiction', 'Romance', 'African American', 'Ch..."
168973,250764,451193911,0,Atlanta Heat,Robert Coram,1997,Signet Book,http://images.amazon.com/images/P/0451193911.0...,"['Romance', 'Romantic Suspense', 'Contemporary..."
168974,250764,451400763,0,Quest,Richard Ben Sapir,1988,New Amer Library (Mm),http://images.amazon.com/images/P/0451400763.0...,"['Science Fiction', 'Fiction', 'Science Fictio..."


In [16]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [17]:
number_rating

,title,rating
0,'Salem's Lot,54
1,...And Ladies of the Club,3
2,"1,000 Places to See Before You Die",10
3,10 Secrets for Success and Inner Peace,2
4,100 Selected Poems,2
...,...,...
8759,Zero at the Bone,28
8760,Zodiac,22
8761,Zombie,7
8762,Zoya's Story: An Afghan Woman's Struggle for F...,9


In [18]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [19]:
number_rating

,title,num_of_rating
0,'Salem's Lot,54
1,...And Ladies of the Club,3
2,"1,000 Places to See Before You Die",10
3,10 Secrets for Success and Inner Peace,2
4,100 Selected Poems,2
...,...,...
8759,Zero at the Bone,28
8760,Zodiac,22
8761,Zombie,7
8762,Zoya's Story: An Afghan Woman's Struggle for F...,9


In [20]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [21]:
final_rating

,user_id,isbn,rating,title,author,year,publisher,image_url,genres,num_of_rating
0,276925,140154078,6,The Music of Chance,Paul Auster,1993,Penguin Books,http://images.amazon.com/images/P/0140154078.0...,"['Fiction', 'American', 'Literature', 'Novels'...",9
1,276925,140327592,0,Matilda,Roald Dahl,1990,Viking Penguin Inc,http://images.amazon.com/images/P/0140327592.0...,"['Childrens', 'Fiction', 'Classics', 'Fantasy'...",82
2,276925,151010633,0,Q,Luther Blissett,2004,Harcourt,http://images.amazon.com/images/P/0151010633.0...,"['Historical Fiction', 'Fiction', 'Historical'...",13
3,276925,385504209,8,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,"['Fiction', 'Mystery', 'Thriller', 'Suspense',...",407
4,276925,553275283,8,Love Story,Erich Segal,1988,Bantam Books,http://images.amazon.com/images/P/0553275283.0...,"['Romance', 'Fiction', 'Classics', 'Contempora...",48
...,...,...,...,...,...,...,...,...,...,...
168971,250764,451191692,0,Remote Control,Stephen White,1998,Signet Book,http://images.amazon.com/images/P/0451191692.0...,"['Thriller', 'Fiction', 'Action', 'Military Fi...",25
168972,250764,451192001,0,How Stella Got Her Groove Back,Terry McMillan,1997,Signet Book,http://images.amazon.com/images/P/0451192001.0...,"['Fiction', 'Romance', 'African American', 'Ch...",123
168973,250764,451193911,0,Atlanta Heat,Robert Coram,1997,Signet Book,http://images.amazon.com/images/P/0451193911.0...,"['Romance', 'Romantic Suspense', 'Contemporary...",3
168974,250764,451400763,0,Quest,Richard Ben Sapir,1988,New Amer Library (Mm),http://images.amazon.com/images/P/0451400763.0...,"['Science Fiction', 'Fiction', 'Science Fictio...",3


In [22]:
final_rating1 = final_rating[final_rating['num_of_rating'] >= 50]

In [23]:
final_rating1

,user_id,isbn,rating,title,author,year,publisher,image_url,genres,num_of_rating
1,276925,140327592,0,Matilda,Roald Dahl,1990,Viking Penguin Inc,http://images.amazon.com/images/P/0140327592.0...,"['Childrens', 'Fiction', 'Classics', 'Fantasy'...",82
3,276925,385504209,8,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,"['Fiction', 'Mystery', 'Thriller', 'Suspense',...",407
9,276925,804106304,0,The Joy Luck Club,Amy Tan,1994,Prentice Hall (K-12),http://images.amazon.com/images/P/0804106304.0...,"['Fiction', 'Historical Fiction', 'Classics', ...",282
10,276929,380807343,0,Coraline,Neil Gaiman,2003,HarperTrophy,http://images.amazon.com/images/P/0380807343.0...,"['Fantasy', 'Horror', 'Fiction', 'Young Adult'...",226
11,276929,380807343,0,Coraline,Neil Gaiman,2003,HarperTrophy,http://images.amazon.com/images/P/0380807343.0...,"['Graphic Novels', 'Fantasy', 'Horror', 'Comic...",226
...,...,...,...,...,...,...,...,...,...,...
168956,250764,446610402,0,Mirage,Don Passman,2001,Warner Vision,http://images.amazon.com/images/P/0446610402.0...,"['Fantasy', 'Dystopia', 'Science Fiction', 'Mi...",130
168964,250764,449221180,0,Native Tongue,Carl Hiaasen,1992,Fawcett Books,http://images.amazon.com/images/P/0449221180.0...,"['Fiction', 'Mystery', 'Humor', 'Crime', 'Thri...",78
168965,250764,449221180,0,Native Tongue,Carl Hiaasen,1992,Fawcett Books,http://images.amazon.com/images/P/0449221180.0...,"['Science Fiction', 'Fiction', 'Feminism', 'Dy...",78
168970,250764,451191013,0,The Regulators,Stephen King,2002,Signet Book,http://images.amazon.com/images/P/0451191013.0...,"['Horror', 'Fiction', 'Thriller', 'Fantasy', '...",93


In [24]:
final_rating1.drop_duplicates(['user_id','title'],inplace=True)

C:\Users\saroj\AppData\Local\Temp\ipykernel_4600\2881104077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating1.drop_duplicates(['user_id','title'],inplace=True)


In [25]:
final_rating1

,user_id,isbn,rating,title,author,year,publisher,image_url,genres,num_of_rating
1,276925,140327592,0,Matilda,Roald Dahl,1990,Viking Penguin Inc,http://images.amazon.com/images/P/0140327592.0...,"['Childrens', 'Fiction', 'Classics', 'Fantasy'...",82
3,276925,385504209,8,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,"['Fiction', 'Mystery', 'Thriller', 'Suspense',...",407
9,276925,804106304,0,The Joy Luck Club,Amy Tan,1994,Prentice Hall (K-12),http://images.amazon.com/images/P/0804106304.0...,"['Fiction', 'Historical Fiction', 'Classics', ...",282
10,276929,380807343,0,Coraline,Neil Gaiman,2003,HarperTrophy,http://images.amazon.com/images/P/0380807343.0...,"['Fantasy', 'Horror', 'Fiction', 'Young Adult'...",226
13,277157,312874243,0,The Return,Buzz Aldrin,2000,St Martins Pr,http://images.amazon.com/images/P/0312874243.0...,"['Historical Fiction', 'Fiction', 'Spain', 'Hi...",126
...,...,...,...,...,...,...,...,...,...,...
168944,250764,446364673,0,The Gift,Kirk Douglas,1993,Warner Books,http://images.amazon.com/images/P/0446364673.0...,"['Fantasy', 'Young Adult', 'Dystopia', 'Fictio...",1216
168952,250764,446610402,0,Mirage,Don Passman,2001,Warner Vision,http://images.amazon.com/images/P/0446610402.0...,"['Romantic', 'Erotica', 'Romance']",130
168964,250764,449221180,0,Native Tongue,Carl Hiaasen,1992,Fawcett Books,http://images.amazon.com/images/P/0449221180.0...,"['Fiction', 'Mystery', 'Humor', 'Crime', 'Thri...",78
168970,250764,451191013,0,The Regulators,Stephen King,2002,Signet Book,http://images.amazon.com/images/P/0451191013.0...,"['Horror', 'Fiction', 'Thriller', 'Fantasy', '...",93


In [26]:
book_pivot = final_rating1.pivot_table(columns='user_id', index='title', values= 'rating')

In [27]:
book_pivot

user_id,183,243,254,507,638,741,882,929,1025,1211,...,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010: Odyssey Two,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,0.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Word of Honor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year of Wonders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
book_pivot.fillna(0, inplace=True)

In [29]:
book_pivot

user_id,183,243,254,507,638,741,882,929,1025,1211,...,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Word of Honor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
